# Parse Apache Log

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark import SQLContext

In [2]:
sc = SparkContext()
spark = SparkSession(sc)

In [3]:
accessLog = sc.textFile('/user/pradeep.s.rajpoot_gmail/access')

In [4]:
import re
row_pattern = re.compile('^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)(.*)" (\d{3}) (\S+)')
def parseLogLine(x):
    m = row_pattern.match(x)
    if m:
        tokens = m.group(4).split(':')
        tFrame = tokens[0] + ':' + tokens[1]
 
        #return { "host" : m.group(1), "timeStamp" : m.group(4), "url" : m.group(6), "httpCode" : int(m.group(8)) }
        return Row(host=m.group(1), timeStamp=m.group(4), timeFrame=tFrame, url=m.group(6), httpCode=int(m.group(8)))
    else:
        #return {}
        return Row(host=None, timeStamp=None, timeFrame = None, url=None, httpCode=None)

In [5]:
log_df = accessLog.map(parseLogLine).toDF()

In [6]:
log_df.head()

Row(host=u'in24.inetnebr.com', httpCode=200, timeFrame=u'01/Aug/1995:00', timeStamp=u'01/Aug/1995:00:00:01 -0400', url=u'/shuttle/missions/sts-68/news/sts-68-mcc-05.txt')

In [7]:
log_df.printSchema()

root
 |-- host: string (nullable = true)
 |-- httpCode: long (nullable = true)
 |-- timeFrame: string (nullable = true)
 |-- timeStamp: string (nullable = true)
 |-- url: string (nullable = true)



In [8]:
accessLog

/user/pradeep.s.rajpoot_gmail/access MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
accessLog.take(2)

[u'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 u'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0']

# Question 1 

Write spark code( using RDD) to find out top 10 requested URLs along with count of number of times they have been requested (This information will help company to find out most popular pages and how frequently they are accessed)

In [10]:
urls = log_df.rdd.filter(lambda x: x['url'] != None).map(lambda x : x.url).countByValue()
#urls = accessLog.map(lambda x : x.split(' ')[6]).countByValue()
sorted_urls = sorted([ (v, k) for (k,v) in urls.items()], reverse=True)
print ('URL'.rjust(50) + 'Count'.rjust(10))
print
for x in sorted_urls[0:10]:
    print (str(x[1]).rjust(50) + str(x[0]).rjust(10))

                                               URL     Count

                        /images/NASA-logosmall.gif     97410
                         /images/KSC-logosmall.gif     75337
                      /images/MOSAIC-logosmall.gif     67448
                         /images/USA-logosmall.gif     67068
                       /images/WORLD-logosmall.gif     66444
                        /images/ksclogo-medium.gif     62778
                                         /ksc.html     43687
           /history/apollo/images/apollo-logo1.gif     37826
                           /images/launch-logo.gif     35138
                                                 /     30347


In [ ]:
#hosts = accessLog.map(lambda x : x.split(' ')[0]).countByValue()
#sorted_hosts = sorted([ (v, k) for (k,v) in hosts.items()], reverse=True)
#print ('IP'.rjust(50) + 'Count'.rjust(10))
#print
#for x in sorted_hosts[0:5]:
#    print (str(x[1]).rjust(50) + str(x[0]).rjust(10))

# Question 2

Write spark code to find out top 5 hosts / IP making the request along with count (This information will help company to find out locations where website is popular or to figure out potential DDoS attacks)

In [66]:
log_df.groupBy('host').count().orderBy('count', ascending=False).show(5)

+--------------------+-----+
|                host|count|
+--------------------+-----+
|  edams.ksc.nasa.gov| 6530|
|piweba4y.prodigy.com| 4846|
|        163.206.89.4| 4791|
|piweba5y.prodigy.com| 4607|
|piweba3y.prodigy.com| 4416|
+--------------------+-----+
only showing top 5 rows



# Question 3

Write spark code to find out top 5 time frame for high traffic (which day of the week or hour of the day receives peak traffic, this information will help company to manage resources for handling peak traffic load)

In [18]:
log_df.groupBy('timeFrame').count().orderBy('count', ascending=False).show(5)

+--------------+-----+
|     timeFrame|count|
+--------------+-----+
|31/Aug/1995:11| 6319|
|31/Aug/1995:10| 6283|
|31/Aug/1995:13| 5948|
|30/Aug/1995:15| 5919|
|31/Aug/1995:09| 5627|
+--------------+-----+
only showing top 5 rows



# Question 4

Write spark code to find out 5 time frames of least traffic (which day of the week or hour of the day receives least traffic, this information will help company to do production deployment in that time frame so that less number of users will be affected if some thing goes wrong during deployment)

In [19]:
log_df.groupBy('timeFrame').count().orderBy('count', ascending=True).show(5)

+--------------+-----+
|     timeFrame|count|
+--------------+-----+
|03/Aug/1995:04|   16|
|03/Aug/1995:09|   22|
|03/Aug/1995:05|   43|
|03/Aug/1995:10|   57|
|03/Aug/1995:07|   58|
+--------------+-----+
only showing top 5 rows



# Question 5

Write spark code to find out unique HTTP codes returned by the server along with count (this information is helpful for devops team to find out how many requests are failing so that appropriate action can be taken to fix the issue)

In [16]:
log_df.filter(log_df.httpCode.isNotNull()).groupBy('httpCode').count().orderBy('count', ascending=False).show()

+--------+-------+
|httpCode|  count|
+--------+-------+
|     200|1398988|
|     304| 134146|
|     302|  26444|
|     404|  10056|
|     403|    171|
|     501|     27|
|     500|      3|
+--------+-------+

